In [24]:
import pymysql

def connect(**kwargs):
    global db  # 提升为全局变量
    db = pymysql.Connect(**kwargs)
    print('连接数据库成功!')

def save(id,name,age):
    pass

def create_table(table_name, *args):
    cursor = db.cursor()  # 打开数据库的游标
    columns = ('%s,'*len(args))[:-1]
    args = (table_name, *args)
    sql = "create table %s("+columns+")"
    cursor.execute(sql % args)
    print('%s 创建成功' % table_name)
    

def query(**kwargs):
    # 可以依据： id, name
    pass

db_config = {
    'host': '127.0.0.1',
    'port': 3306,
    'user':'root',
    'password':'root',
    'db':'xa_py3',
    'charset':'utf8'
}
connect(**db_config)  # 提取字典的数据，转成key=value
create_table('t_qx', 'id int', 'title varchar(20)')

连接数据库成功!
t_qx 创建成功


In [25]:
def fx(a):
    fx.a += a  # fx.a 是函数fx对象的属性
    print(fx.a)

In [27]:
fx.a = 120
fx(100)
type(fx)

220


function

In [30]:
# 无参的装饰函数
import time

def test_runtime(fun):
    print('--A---')
    def wrapper(*args, **kwargs):
        # 计算函数运行的时间
        print('--AA---')
        start_time = time.time()
        result = fun(*args, **kwargs)
        used_time = time.time()-start_time
        print('%s 用时 %.4f 秒' % (fun.__name__, used_time))
        return result
    
    return wrapper

@test_runtime
def main():
    connect(**db_config)

main()

--A---
--AA---
连接数据库成功!
main 用时 0.0027 秒


In [37]:
# 有参数的装饰函数
rights = {'QX_ADD': 1,
          'QX_EDIT': 2,
          'QX_DEL': 4}

user_qx = 6  # 当前用户的权限

def check_qx(qx_name):
    print('--A--')
    def wrapper1(fun):
        print('--B--')
        def wrapper2(*args, **kwargs):
            print('---C--')
            # 判断当前用户的权限是否具有qx_name 
            qx_right = rights.get(qx_name)
            if all((qx_right, user_qx & qx_right == qx_right)):
                # 有权限
                result = fun(*args, **kwargs)
                return result
            else:
                print('－－无此权限－－')
        
        return wrapper2
    
    return wrapper1


@check_qx('QX_ADD')  # 检查当前用户是否拥有add权限
def add(id, title):
    connect(**db_config)
    cursor = db.cursor()
    cursor.execute('insert t_qx(id,title) values(%s,%s)',
                   (id, title))
    db.commit() # 提交
    print('增加 t_qx 成功!')
    
add(2, '修改')

--A--
--B--
---C--
－－无此权限－－


In [41]:
# 无参的装饰器类
import time

class TestRuntime():
    def __init__(self, fun):
        self.fun = fun
        
    def __call__(self, *args, **kwargs):
         # 计算函数运行的时间
        start_time = time.time()
        result = self.fun(*args, **kwargs)
        used_time = time.time()-start_time
        print('%s 用时 %.4f 秒' % (self.fun.__name__, used_time))
        return result


@TestRuntime
def main():
    connect(**db_config)

main()

--AA---
连接数据库成功!
main 用时 0.0031 秒


In [43]:
class CheckQX():
    def __init__(self, qx_name):
        self.qx_name = qx_name
        
    # __call__ 在当前类的实例对象作为函数被调用时，会调用__call__函数
    def __call__(self, fun):
        def wrapper(*args, **kwargs):
            # 判断当前用户的权限是否具有qx_name 
            qx_right = rights.get(self.qx_name)
            if all((qx_right, user_qx & qx_right == qx_right)):
                # 有权限
                result = fun(*args, **kwargs)
                return result
            else:
                print('－－无此权限－－')
                
        return wrapper


@CheckQX('QX_ADD')  # 检查当前用户是否拥有add权限
def add(id, title):
    connect(**db_config)
    cursor = db.cursor()
    cursor.execute('insert t_qx(id,title) values(%s,%s)',
                   (id, title))
    db.commit() # 提交
    print('增加 t_qx 成功!')

user_qx = 3  # 当前用户的权限
add(3, '删除')

连接数据库成功!
增加 t_qx 成功!


In [44]:
class Person():
    def __init__(self, name):
        self.name = name
    
    def __call__(self):
        print('当前人员的姓名',self.name)
        
p = Person('Disen')
p()  # 实例对象当作函数被调用

当前人员的姓名 Disen


In [48]:
# 扩展 python上下文 context
class DB():
    def __init__(self, **kwargs):
        self.host = kwargs.get('host')
        self.port = kwargs.get('port')
        self.db = kwargs.get('db')
        self.user = kwargs.get('user')
        self.password = kwargs.get('password')
        self.charset = kwargs.get('charset')
    
    def __enter__(self):
        # 打开连接连接
        self.db = pymysql.Connect(host=self.host,
                                  port=self.port,
                                  db=self.db,
                                  user=self.user,
                                  password=self.password,
                                  charset=self.charset)
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.db.close()  # 关闭数据连接
        
    def query(self):
        cursor = self.db.cursor()
        cursor.execute('select * from t_qx')
        for row in cursor.fetchall():
            print(row)
            
db_ = DB(**db_config)
with db_ :  # 当进入上下文时，调用db_的__enter__函数
    if db_:
        db_.query()
    
# 退出上下文时， 调用db_的__exit__函数


(1, '查询')
(3, '删除')
